In [ ]:
import os
import sqlite3
from datetime import datetime, timedelta

### Creating the database for file metadata

In [ ]:

# Initialize SQLite database
conn = sqlite3.connect('metadata.db')
cursor = conn.cursor()

# Create table to store metadata
cursor.execute('''
CREATE TABLE IF NOT EXISTS file_metadata (
    path TEXT PRIMARY KEY,
    size INTEGER,
    creation_time TEXT,
    last_modified_time TEXT
)''')

# Commit changes
conn.commit()


In [ ]:

def gather_metadata(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            file_path = os.path.join(root, file)
            
            # Extract metadata
            file_stat = os.stat(file_path)
            size = file_stat.st_size
            creation_time = file_stat.st_ctime
            last_modified_time = file_stat.st_mtime
            
            # Insert metadata into database
            cursor.execute('''
            INSERT OR REPLACE INTO file_metadata
            (path, size, creation_time, last_modified_time)
            VALUES (?, ?, ?, ?)''', (file_path, size, creation_time, last_modified_time))
    
    # Commit changes to database
    conn.commit()


In [ ]:

# Gather metadata starting from root folder
gather_metadata('/mn/stornext/d12/lapalma/reduc/2023/2023-05-03')

# Query example: fetch all files larger than 1MB
cursor.execute('SELECT * FROM file_metadata WHERE size > 1048576')
print(cursor.fetchall())

# Close the database connection
conn.close()


### Working with the metadata

In [ ]:

# Connect to SQLite database
conn = sqlite3.connect('metadata.db')
cursor = conn.cursor()

# Query the table schema
cursor.execute("PRAGMA table_info(file_metadata)")
columns = cursor.fetchall()

# Print metadata types
print("Available Metadata Types:")
for column in columns:
    print(f"{column[1]} ({column[2]})")

# Close the database connection
conn.close()


In [ ]:
def filter_metadata(created_after=None, modified_within_months=None, min_size_gb=None, name_contains_list=None, extensions=None):
    # Initialize SQL query and parameters list
    query = "SELECT * FROM file_metadata WHERE "
    params = []
    
    # Add conditions based on provided arguments
    conditions = []
    
    if created_after:
        timestamp = datetime.timestamp(datetime.strptime(created_after, "%Y-%m-%d"))
        conditions.append("creation_time > ?")
        params.append(timestamp)
    
    if modified_within_months:
        past_timestamp = datetime.timestamp(datetime.now() - timedelta(days=modified_within_months*30))
        conditions.append("last_modified_time > ?")
        params.append(past_timestamp)
        
    if min_size_gb:
        min_size_bytes = min_size_gb * (1024 ** 3)
        conditions.append("size > ?")
        params.append(min_size_bytes)
        
    if name_contains_list:
        name_conditions = []
        for substring in name_contains_list:
            name_conditions.append("path LIKE ?")
            params.append(f"%{substring}%")
        conditions.append("(" + " AND ".join(name_conditions) + ")")
    
    if extensions:
        extension_conditions = []
        for ext in extensions:
            extension_conditions.append("path LIKE ?")
            params.append(f"%.{ext}")
        conditions.append("(" + " OR ".join(extension_conditions) + ")")
        
    # Finalize query
    query += " AND ".join(conditions)
    
    # Execute query
    conn = sqlite3.connect('metadata.db')
    cursor = conn.cursor()
    cursor.execute(query, params)
    records = cursor.fetchall()
    
    # Close the database connection
    conn.close()
    
    return records

In [ ]:
def human_readable_size(size, decimal_places=2):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024.0:
            break
        size /= 1024.0
    return f"{size:.{decimal_places}f} {unit}"

def timestamp_to_date(timestamp):
    return datetime.fromtimestamp(float(timestamp)).strftime('%Y-%m-%d %H:%M:%S')

def pretty_print(records):
    print(f"{'Path':<160} {'Size':<12} {'Creation Time':<25} {'Last Modified Time':<25}")
    print("="*220)
    
    for record in records:
        path, size, creation_time, last_modified_time = record
        size_str = human_readable_size(size)
        creation_time_str = timestamp_to_date(creation_time)
        last_modified_time_str = timestamp_to_date(last_modified_time)
        print(f"{path:<160} {size_str:<12} {creation_time_str:<25} {last_modified_time_str:<25}")


In [ ]:

# Example usage
filtered_records = filter_metadata(
    created_after="2023-05-02",
    modified_within_months=6,
    min_size_gb=10,
    name_contains_list=["2023-05-03T09:00:41", "corrected_im"],
    extensions=["fits", "icube"]
)

print("Filtered Records:")
pretty_print(filtered_records)
